In [1]:
import sys
import os
import shutil

In [2]:
sys.path.append('..')

In [3]:
from jamo import j2hcj, h2j
import pandas as pd
import numpy as np
from konlpy.tag import Twitter

from sklearn.metrics import accuracy_score, f1_score
twitter = Twitter()

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
from utils.prepare_data import *
import time
from utils.model_helper import *

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
from custum_dataset import digi_data_load, testcase_add_data_load, testcase_shuffle_data_load
from attn_bi_lstm import Shared_ABLSTM

In [6]:
_, _, _, _, cols = digi_data_load()

4244 1062


In [7]:
onehot_mat = pd.get_dummies(cols).astype(float)
onehot_mat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 33 columns):
DIGITAL_Pa/s문의          33 non-null float64
DIGITAL_P가격문의           33 non-null float64
DIGITAL_P결제문의           33 non-null float64
DIGITAL_P교환문의           33 non-null float64
DIGITAL_P기타faq문의        33 non-null float64
DIGITAL_P대화종료           33 non-null float64
DIGITAL_P도움말보기          33 non-null float64
DIGITAL_P반품문의           33 non-null float64
DIGITAL_P배송문의           33 non-null float64
DIGITAL_P배송일문의          33 non-null float64
DIGITAL_P배송지변경문의        33 non-null float64
DIGITAL_P본인인증문의         33 non-null float64
DIGITAL_P상담시간           33 non-null float64
DIGITAL_P상담요청           33 non-null float64
DIGITAL_P상담원연결          33 non-null float64
DIGITAL_P상담품목           33 non-null float64
DIGITAL_P상품문의_P상담원연결    33 non-null float64
DIGITAL_P전화요청           33 non-null float64
DIGITAL_P주문이력확인         33 non-null float64
DIGITAL_P주문취소           33 non-null float64
DIGITAL_P차이점     

In [8]:
df = pd.read_csv('../../nlu-model-analysis/binary_dataset.csv',sep='\t',index_col='Unnamed: 0')
df = df.sample(len(df),random_state=10)
df['binary_class'].value_counts()

0    60407
1    47390
Name: binary_class, dtype: int64

In [9]:
def class2onehot(label):
    if label not in onehot_mat.columns.tolist():
        return np.zeros((len(onehot_mat),))
    
    return onehot_mat[label].values.reshape(-1,)

In [10]:
train_df = df.iloc[:int(len(df)*0.8)]
test_df = df.iloc[int(len(df)*0.8):]

In [11]:
TRAIN_DOC, TRAIN_LABEL, TRAIN_BIN_LABEL, TEST_DOC, TEST_LABEL, TEST_BIN_LABEL = \
    train_df['sent'], train_df['class'], train_df['binary_class'], \
    test_df['sent'], test_df['class'], test_df['binary_class']

In [12]:
train_intent = np.asarray(TRAIN_LABEL.apply(lambda x: class2onehot(x).tolist()).tolist())
test_intent = np.asarray(TEST_LABEL.apply(lambda x: class2onehot(x).tolist()).tolist())

In [13]:
print(train_intent.shape, test_intent.shape)

(86237, 33) (21560, 33)


In [14]:
dis_train = TRAIN_BIN_LABEL
dis_test = TEST_BIN_LABEL

In [15]:
%%time
train_doc = TRAIN_DOC.tolist()
test_doc = TEST_DOC.tolist()

#train_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), train_doc)))
#test_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), test_doc)))
train_doc = pd.Series(list(map(lambda x: ' '.join(list(x.replace(' ','ⓢ'))), train_doc)))
test_doc = pd.Series(list(map(lambda x: ' '.join(list(x.replace(' ','ⓢ'))), test_doc)))

CPU times: user 207 ms, sys: 6.89 ms, total: 214 ms
Wall time: 212 ms


In [16]:
y_train = train_intent
y_test = test_intent

In [17]:
x_train, x_test, tokenizer = \
        data_preprocessing_v2(train_doc, test_doc, max_len=64)
vocab_size = tokenizer.num_words

In [18]:
tokenizer.word_docs

{'플': 878,
 '이': 18457,
 '레': 1484,
 '디': 2779,
 '스': 6841,
 '로': 6714,
 '우': 1820,
 '렐': 20,
 '코': 950,
 '나': 8175,
 '폰': 3697,
 '안': 5265,
 '되': 4566,
 '요': 22137,
 '쿠': 3220,
 'ⓢ': 52623,
 '헤': 262,
 '어': 12477,
 '장': 3556,
 '식': 907,
 '현': 539,
 '영': 539,
 '수': 4232,
 '발': 1000,
 '필': 751,
 '증': 1099,
 '금': 2371,
 '행': 774,
 '해': 6880,
 '3': 2718,
 '8': 1942,
 '6': 1897,
 'b': 904,
 '0': 5287,
 'h': 1236,
 '니': 7587,
 '다': 8135,
 '하': 9381,
 '에': 6687,
 '담': 1356,
 '습': 1773,
 '겠': 576,
 '주': 6737,
 '세': 4819,
 '려': 3244,
 '알': 1957,
 '치': 2188,
 '약': 482,
 '아': 8703,
 '동': 1412,
 '닥': 95,
 '방': 1993,
 '가': 12256,
 '탈': 254,
 '것': 1040,
 '는': 11737,
 '곳': 169,
 '여': 3017,
 '기': 8156,
 '뭐': 1242,
 'n': 2668,
 '줄': 296,
 '무': 1994,
 '고': 10140,
 '씨': 321,
 '옹': 42,
 'ㅓ': 70,
 '오': 3938,
 '청': 2719,
 '결': 2078,
 '제': 7681,
 '성': 1935,
 '양': 626,
 '손': 165,
 '톱': 26,
 '게': 4486,
 '교': 744,
 '환': 2425,
 '떻': 2150,
 '은': 5570,
 '구': 5439,
 '유': 1803,
 '트': 6072,
 '르': 692,
 '시': 4521,
 '

In [19]:
config = {
    "max_len": 64,
    "hidden_size": 64,
    "vocab_size": vocab_size,
    "embedding_size": 128,
    "n_class": 33,
    "learning_rate": 1e-3,
    "decay": 0.90,
    "batch_size": 512,
    "train_epoch": 20,
    "dis_class": 2
}

In [20]:
modelname = 'shared_binary_classifier'

try:
    shutil.rmtree('./output/'+modelname)
except OSError as e:
    if e.errno == 2:
        # 파일이나 디렉토리가 없음!
        print ('No such file or directory to remove')
        pass
    else:
        raise

In [21]:
sess = tf.Session()

classifier = Shared_ABLSTM(sess, config)
classifier.build_graph(modelname)

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

resume = False
if resume:
    saver.restore(sess, "./Saver/{}/{}.ckpt".format(modelname, modelname))

start = time.time()
step = 0
lr = config['learning_rate']

building graph
graph built successfully!


In [ ]:
for e in range(config["train_epoch"]):
    
    t0 = time.time()
    print("Epoch %d start !" % (e + 1))
    for x_batch, y_batch, dis_batch in shared_fill_feed_dict(x_train, y_train, dis_train, config["batch_size"]):
        _, global_step = sess.run([classifier.train_op, classifier.global_step],
                feed_dict={classifier.x:x_batch,
                           classifier.label:y_batch,
                           classifier.dis_label:dis_batch,
                           classifier.keep_prob:0.5,
                           classifier.LEARNING_RATE: lr})
    
        if step % 10 == 0:
            train_loss, act_train_loss, dis_train_loss, dis_acc = sess.run([classifier.loss, 
                                                                   classifier.n_loss,
                                                                   classifier.dis_loss,
                                                                   classifier.dis_accuracy],
                                                                             feed_dict={
                                                                                classifier.x:x_batch,
                                                                                classifier.label:y_batch,
                                                                                classifier.dis_label:dis_batch,
                                                                                classifier.keep_prob:1})
            print(train_loss, act_train_loss, dis_train_loss, dis_acc)
            
#             feed_dict = make_test_feed_dict(classifier, (x_batch, y_batch))
#             tr_merged = sess.run(classifier.merge, feed_dict)
#             feed_dict = make_test_feed_dict(classifier, (x_test, y_test))
#             ts_merged = sess.run(classifier.merge, feed_dict)
#             classifier.train_writer.add_summary(tr_merged, step)
#             classifier.test_writer.add_summary(ts_merged, step)
            
        step += 1
    t1 = time.time()
    if e+1 % 10: 
        lr = lr*config['decay']
#     print("Train Epoch time:  %.3f s" % (t1 - t0))
#     dev_acc = run_eval_step(classifier, sess, dev_batch)
#     print("validation accuracy: %.3f " % dev_acc)

Epoch 1 start !
4.1161194 3.4312716 0.6848477 0.57421875
3.7831717 3.180651 0.6025208 0.7109375
3.7023258 3.162027 0.54029906 0.7363281
3.6917286 3.1445386 0.54719007 0.7246094
3.6009374 3.092556 0.5083813 0.76171875
3.5927413 3.1428874 0.4498539 0.8203125
3.5187483 3.0957391 0.4230091 0.8222656
3.4882007 3.1002297 0.3879708 0.8378906
3.5240963 3.12555 0.3985462 0.8261719
3.5221777 3.126871 0.39530665 0.828125
3.4145331 3.0402753 0.37425777 0.8339844
3.4546866 3.114503 0.34018368 0.8613281
3.395953 3.0798032 0.31614968 0.875
3.4525714 3.1084487 0.34412265 0.8535156
3.4404883 3.144359 0.2961293 0.890625
3.5052457 3.147007 0.35823882 0.8457031
3.4016187 3.063642 0.3379767 0.8730469
Epoch 2 start !
3.3574584 3.0668068 0.2906516 0.8925781
3.327657 3.0253212 0.30233583 0.87109375
3.378421 3.039129 0.3392921 0.8671875
3.427639 3.1228426 0.30479658 0.8886719
3.4689455 3.149641 0.3193044 0.87890625
3.4561205 3.1791694 0.27695107 0.90234375
3.4161491 3.1369069 0.27924216 0.9003906
3.3463397 3.0

In [17]:
if "Saver" not in os.listdir("./"):
    os.makedirs("./Saver")
if modelname not in os.listdir("./Saver"):
    os.makedirs("./Saver/{}".format(modelname))
saver.save(sess, "./Saver/{}/{}.ckpt".format(modelname, modelname))

'./Saver/binary_classifier/binary_classifier.ckpt'

In [18]:
print("Training finished, time consumed : ", time.time() - start, " s")
print("Start evaluating:  \n")
cnt = 0
test_acc = 0
for x_batch, y_batch in fill_feed_dict(x_test, y_test, config["batch_size"]):
    acc = run_eval_step(classifier, sess, (x_batch, y_batch))
    test_acc += acc
    cnt += 1

print("Test accuracy : %f %%" % (test_acc / cnt * 100))

Training finished, time consumed :  5620.205155849457  s
Start evaluating:  

Test accuracy : 87.662760 %
